In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns
import sklearn.datasets

In [98]:

train_data = pd.read_csv(r'C:\Users\lscon\Desktop\AA\projeto\the-three-body-problem\mlNOVA\X_train.csv')
#test_data = pd.read_csv(r'C:/Users/duart/OneDrive/Ambiente_de_Trabalho/Master_Analysis_Engineering_Big_Data/23-24/1st_semester/AA_ML/Kaggle_challenges/3_body_problem/3_body_problem/X_test.csv')

#Definir as variáveis
t = pd.DataFrame(train_data, columns=['t'])
x1 = pd.DataFrame(train_data, columns=['x_1'])
x2 = pd.DataFrame(train_data, columns=['x_2'])
x3 = pd.DataFrame(train_data, columns=['x_3'])
y1 = pd.DataFrame(train_data, columns=['y_1'])
y2 = pd.DataFrame(train_data, columns=['y_2'])
y3 = pd.DataFrame(train_data, columns=['y_3'])
vx1 = pd.DataFrame(train_data, columns=['v_x_1'])
vx2 = pd.DataFrame(train_data, columns=['v_x_2'])
vx3 = pd.DataFrame(train_data, columns=['v_x_3'])
vy1 = pd.DataFrame(train_data, columns=['v_y_1'])
vy2 = pd.DataFrame(train_data, columns=['v_y_2'])
vy3 = pd.DataFrame(train_data, columns=['v_y_3'])

In [ ]:
# Identify faulty rows based on the criterion (all values = 0.0 except for Id)
zero_rows = train_data[(train_data.drop('Id', axis=1) == 0).all(axis=1)]

# Remove the faulty rows from the DataFrame
train_data_preprocessed = train_data[~train_data.index.isin(zero_rows.index)]
train_data_preprocessed.reset_index(drop=True, inplace=True)
#train_data_preprocessed.to_csv('train_preprocessed.csv', index=False)

In [ ]:
#gives the stats for the preprocessed data (without the rows with zeros)
summary_stats_filtered = train_data_preprocessed.describe(include='all')

#gives the stats for the nonprocessed data
summary_stats = train_data.describe(include="all")

# Test...

In [ ]:
# Calculate the correlation matrix and plot it as a heatmap
corr_matrix = train_data_preprocessed.drop(train_data_preprocessed.columns[13],
                                       axis=1).corr()
corr_matrix.to_excel('corr_matrix_train_processed.xlsx')

plt.figure(figsize=(20, 16), dpi=800)
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.4f',
            linewidths=0.5)
plt.title('Correlation Matrix Heatmap')
plt.savefig('Corr_matrix_heatmap.jpg', dpi=800)
plt.show()

# Create a pairwise scatter plot matrix
scatter_matrix = pd.plotting.scatter_matrix(train_data_preprocessed,
                                            figsize=(20, 20))
plt.show()


In [ ]:
#create pairwise plots of correlation between variables

rows = 50000
x1_rows = x1.head(n=rows)
partial_train_data = train_data_preprocessed.drop(columns=['v_x_1','v_x_2','v_y_1', 
                                                           'v_y_2', 'v_x_3', 'v_y_3']).head(n=rows)


_= sns.pairplot(partial_train_data, kind="reg", diag_kind="kde", plot_kws={'line_kws':{'color':'red'}})
plt.title('Pairwise plots t vs velocity components')
plt.savefig('pairwisetv_50000')

In [97]:
#create feature label matrices
#we're not going to use the velocity components as features
train_data_without_velocity = train_data_preprocessed.drop(columns=['Id','v_x_1','v_x_2','v_y_1', 
                                                           'v_y_2', 'v_x_3', 'v_y_3'])
#divide by simulations
list_of_times = [values for values in train_data_without_velocity['t']]
time_index_tuples = list(enumerate(list_of_times))
zeros_indexes = list(filter(lambda value: value[1] == 0, time_index_tuples))
zeros_indexes = [value[0] for value in zeros_indexes] 
list_of_simulations = []
lower_bound = 0
for i in range(len(zeros_indexes)-1):
    simulation = train_data_without_velocity.iloc[lower_bound:zeros_indexes[i+1]]
    list_of_simulations.append(simulation)
    lower_bound = zeros_indexes[i+1]
from tqdm import tqdm

#add label and put the starting position at every row
#x1
for simulation in tqdm(list_of_simulations):
    first_row_values = simulation.head(1)
    simulation.loc[:,'x_1_label'] = simulation.loc[:, 'x_1']
    for index, row in simulation.iterrows():
        simulation.at[index, 'x_1'] = first_row_values['x_1']
        simulation.at[index, 'y_1'] = first_row_values['y_1']
        simulation.at[index, 'x_2'] = first_row_values['x_2']
        simulation.at[index, 'y_2'] = first_row_values['y_2']
        simulation.at[index, 'x_3'] = first_row_values['x_3']
        simulation.at[index, 'y_3'] = first_row_values['y_3']

list_of_simulations.to_csv('feature_matrice_x1.csv', index=False)


  0%|          | 0/4997 [00:00<?, ?it/s]C:\Users\lscon\AppData\Local\Temp\ipykernel_6400\2281588060.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simulation.loc[:,'x_1_label'] = simulation.loc[:, 'x_1']
C:\Users\lscon\AppData\Local\Temp\ipykernel_6400\2281588060.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simulation.loc[:,'x_1_label'] = simulation.loc[:, 'x_1']
C:\Users\lscon\AppData\Local\Temp\ipykernel_6400\2281588060.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of

             t       x_1       y_1       x_2       y_2       x_3       y_3  \
0     0.000000  1.000000  0.000000 -0.266467  0.859196 -0.733533 -0.859196   
1     0.039062  0.999548  0.000092 -0.266261  0.858781 -0.733287 -0.858874   
2     0.078125  0.998190  0.000370 -0.265641  0.857535 -0.732549 -0.857905   
3     0.117188  0.995925  0.000833 -0.264606  0.855456 -0.731318 -0.856289   
4     0.156250  0.992747  0.001483 -0.263154  0.852540 -0.729592 -0.854022   
..         ...       ...       ...       ...       ...       ...       ...   
252   9.843750  1.471277  1.133103  0.966676  1.248312 -2.437953 -2.381415   
253   9.882810  1.483859  1.154101  0.978161  1.249428 -2.462019 -2.403529   
254   9.921880  1.490718  1.176133  0.995286  1.249430 -2.486004 -2.425563   
255   9.960940  1.491492  1.199018  1.018416  1.248499 -2.509908 -2.447517   
256  10.000000  1.485476  1.222565  1.048258  1.246830 -2.533734 -2.469395   

     x_1_label  
0     1.000000  
1     0.999548  
2     0.9981